In [1]:
import math
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#  gammaN      This function calculates  free ion coefficients for the following neutral
#              solutes:
#
#              NH3,  B(OH)3, H3PO4, H2S, SO2, HF, CO2
#                                 *******************************
#
#                   gN = gammaN(Tc,I,mT_cat,mT_an)
#                           Tc = Temperature in degree celcius  
#                            I = Ionic strength
#                      mT_cat  =   Total concentration (molals) of major 
#                                  cations in seawater (Na, K, Mg, Ca, Sr)
#                       mT_an  =   Total concentration (molals) of major 
#                                  anions in seawater  (Cl, SO4, CO3, HCO3, Br, F, B(OH)4)

## --------------------------------------------------------------------------------------
##  Assign values to global variables
mjCaMax = 5 + 2         #In python array goes 0,1,2,3,4,5,6, so mjCaMax must go to 7 to include the 6th variable
mnCaMax = 37
CaMax = mjCaMax + mnCaMax

mjAnMax = 7 + 1          #In python array goes 0,1,2,3,4,5,6,7,8 so mjCaMax must go to 8 to include the 7th variable
mnAnMax = 19
AnMax = mjAnMax + mnAnMax

NeutMax = 7

## ------------------------------------------------------------------------------

def gammaNV3(Tc, I, mT_cat, mT_an):
    
    T = Tc + 273.15
    veclengths=[size(Tc), size(I), size(len(mT_cat)), size(len(mT_an))]
    maxr = max(veclengths)
    
    if len(unique(veclengths)) > 2:
        print('*** INPUT ERROR: Input vectors must all be of same length, or of length 1. ***')
    
    def lamda(T, p):
        P = p[:,0] + p[:,1]*T + p[:,2]*(T**2) + p[:,3]/T + p[:,4]*log(T)
        return P

    def zeta(T,q):
        Q = np.zeros([2, 5])
        for an in range(2):
            for cat in range(5):
                Q[an, cat] = q[0, an, cat] + q[1, an, cat]*T + q[2, an, cat]*(T**2) + q[3, an, cat]/T + q[4, an, cat]*log(T)
        return Q
            
    ## --------------------------------------------------------------------------------------

    ## LnGamma for Gamma 
    lng_N = np.zeros([maxr,NeutMax])
    
    ## NH3    --> [Clegg & Brimblecombe. 1989]
    lng_N[:,0] = 2*(0.0175* mT_cat[0] + 0.0454* mT_cat[1] - 0.21* mT_cat[2] - 0.081* mT_cat[3] - 0.041* mT_cat[4]) + 2* (0.138* mT_an[1] + 0.174 * mT_an[2]) - (0.00134 * mT_cat[3]* mT_an[0])
    
    ## B(OH)3 
    lng_N[:,1] = 2*(-0.097* mT_cat[0] - 0.14* mT_cat[1]) + 2* (0.091 * mT_an[0] +  0.018* mT_an[1]) + (0.046* mT_an[0]* mT_an[1]) 
    
    ## H3PO4
    lng_N[:,2] = 2*(0.075* mT_cat[0] + 0.075* mT_cat[2] - 0.07* mT_cat[1])    # H3PO4 Na from Hershey et al. 1989? and Mg = Na    % Silvester, Pitzer 1976
    
    ## H2S    --> [Hershey et al. 1998]
    lng_N[:,3] = (0.1554*I) - (0.00806*(I**2))
    
    ## SO2 
    lng_N[:,4] = 2*(0.0283* mT_cat[0] + 0.085* mT_cat[2])
    
    ## HF    
    lng_N[:,5] = 2*0.011* mT_cat[0]
    
    # ****************************************************************************************
    
    ## CO2 
    # lambdaNc: interaction between CO2 and major cations 
    
    #for in = 1:maxr
    
    Tx = T
    Nc = np.zeros([mjCaMax, 5])  # Na, K, Mg, Ca, Sr

    Nc [0,:] = [-5496.38465, -3.326566, 0.0017532, 109399.341, 1047.021567]      # Na
    Nc [1,:] = [2856.528099, 1.7670079, -0.0009487, -55954.1929, -546.074467]    # K
    Nc [2,:] = [-479.362533, -0.541843, 0.00038812, 3589.474052, 104.3452732]    # Mg
    Nc [3,:] = [-12774.6472, -8.101555, 0.00442472, 245541.5435, 2452.50972]     # Ca
    # Nc (5,:) = [0,0,0,0,0] 

    lambdaNc = lamda(Tx, Nc)
    
    # ****************************************************************************************

    # lambdaNa: interaction between CO2 and major anions.
    Na = np.zeros([2,5])           # Cl, SO4

    Na[0]= [1659.944942, 0.9964326, -0.00052122, -33159.6177, -315.827883]    # Cl
    Na[1]= [2274.656591, 1.8270948, -0.00114272, -33927.7625, -457.015738]    # SO4

    lambdaNa = lamda(Tx, Na)

    # ****************************************************************************************

    # ZetaCO2:  triplet interaction between CO2, major cations and major anions.
    
    param_zeta = zeros([5,2,5])     #[set, row, number in row]
    param_zeta[0,0] = [-379.459185,-379.686097,-1342.60256,-166.06529,0]
    param_zeta[0,1] = [67030.02482,-2907.03326,-7374.24392,0,0]      
    param_zeta[1,0] = [-0.258005,-0.257891,-0.772286,-0.018002,0]    
    param_zeta[1,1] = [37.930519,-2.860763,-4.608331,0,0]   
    param_zeta[2,0] = [0.000147823,0.000147333,0.000391603,-0.0000247349,0]     
    param_zeta[2,1] = [-0.0189473,0.001951086,0.002489207,0,0]     
    param_zeta[3,0] = [6879.030871,6853.264129,27726.80974,5256.844332,0]
    param_zeta[3,1] = [-1399082.37,30756.86749,143162.6076,0,0]
    param_zeta[4,0] = [73.74511574,73.79977116,253.62319406,27.377452415,0]
    param_zeta[4,1] = [-12630.27457,611.37560512,1412.302898,0,0]
    #param_zeta(2,6,:) = [-804.121738,-0.470474,0.000240526,16334.38917,152.3838752]   # Cl & H
    
    ZetaCO2 = zeta(Tx, param_zeta)
    
    ## Calculating free activity coefficient of neutral solutes:

    for cat in range(5):
        lng_N[:,6] = lng_N[:,6] + 2*(mT_cat[cat]*lambdaNc[cat])

    for an in range(2):
        lng_N[:,6] = lng_N[:,6] + 2*(mT_an[an]*lambdaNa[an])

    for an in range(2):
        for cat in range(5):
            lng_N[:,6] = lng_N[:,6]+ (mT_an[an]* mT_cat[cat]* ZetaCO2[an,cat]) 

    # gammaCO2gas = math.exp(1/(8.314462175*T*(0.10476 - 61.0102/T -660000/T/T/T -2.47E27/math.pow(T,12))));

    ## LnGamma --> Gamma
    gN = exp(lng_N)

    return gN

In [3]:
gN = gammaNV3(25, 0.7228, [0.4689674, 0.0102077, 0.0528171, 0.0102821, 0.0000907], [0.5458696, 0.0282352, 0.00024694, 0.00179143, 0.0008421, 0.0000683, 0.0001008])
print(gN)

[[ 1.00135665  1.00726213  1.07986777  1.11417281  1.03616092  1.01037069
   1.13323512]]


In [74]:
#Tc = [25,26,27]
Td = np.array([25,26,27])
T = Td + 273.15
maxr = len(Tc)

for i in range(maxr):
    Tx = T[i]

    print(Tx)

298.15
299.15
300.15


In [60]:
k = 27

for i in range(1):
    print(k[i])

TypeError: 'int' object is not subscriptable